In [3]:
#take tags only from test5 database

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter


#df = pd.read_csv('train.csv')

In [72]:
test_data = pd.read_csv('test1_all.csv')

In [73]:
# delete all data with empty tags from test data
test_data.dropna(subset=['photoTags'], inplace=True)

In [74]:
#take tags from the column 'photoTags'
tags = test_data.loc[:,'photoTags']

In [75]:
# join them to list
text = ' '.join(test_data['photoTags'])
text_list = text.split(' ')

In [76]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit_transform(tags)

<4307x10208 sparse matrix of type '<class 'numpy.int64'>'
	with 34706 stored elements in Compressed Sparse Row format>

In [77]:
print(len(vectorizer.vocabulary_))

10208


In [78]:
# save the corpus into the file
import csv

dt = vectorizer.vocabulary_


d = pd.DataFrame.from_dict(dt, orient='index', dtype=None)

d.to_csv("corpus1.csv", encoding="utf-8", header=None)


In [79]:
#LOAD DATA
crp = pd.read_csv("corpus1.csv", header=None)
crp.to_csv("corpus1.csv", encoding="utf-8", header=['words','id'], index=None)

In [80]:
#LOAD DATA
crp = pd.read_csv("corpus1.csv")

#CLEAN DATA
crp['words'] = crp['words'].str.replace("\d+", '')
crp['words'] = crp['words'].str.replace("sec", '')
crp['words'] = crp['words'].str.replace("nabkv", '')
# delete values less then 3
crp =crp[crp['words'].str.len() > 3]
# delete geolat geolon
crp['words'] = crp['words'].str.replace("geolat", '')
crp['words'] = crp['words'].str.replace("geolon", '')
crp['words'] = crp['words'].str.replace("geotagged", '')

#train['words'] = train['words'].str.replace("^[0-9]*$", '')
#s = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", s)
crp['words'].replace('', np.nan, inplace=True)
crp = crp.dropna(subset=['words'])


In [81]:
crp = crp[crp['words'].str.len() > 3]


In [85]:
# save data
crp.to_csv("corpus.csv", encoding="utf-8",index=None)

In [86]:
# load data again 
crp = pd.read_csv('corpus.csv')
#take tags from the column 'photoTags'
tags = crp.loc[:,'words']

In [87]:
# so we have features 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tf_matrix = tfidf.fit_transform(tags)

In [88]:
vocab = tfidf.vocabulary_

In [89]:
# we have sparse matrix for 3 mln pictures 
# create KNN
from sklearn.neighbors import NearestNeighbors
train_data = pd.read_csv('dm_clean3.csv')
train_data_ = train_data.loc[:,'photoTags']
_vect = TfidfVectorizer(vocabulary = vocab)
train_c_matrix = _vect.fit_transform(train_data_)

nbrs = NearestNeighbors(n_neighbors = 1, algorithm='ball_tree')
nbrs.fit(train_c_matrix)

C:\Apps\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:211: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=1, p=2, radius=1.0)

In [90]:
test_data = pd.read_csv('test1_all.csv')
# delete all data with empty tags from test data
#test_data.dropna(subset=['photoTags'], inplace=True)
test_data['photoTags'].replace( np.nan,'', inplace=True)
test_data_ = test_data.loc[:,'photoTags']
new_vect = TfidfVectorizer(vocabulary = vocab)
test_c_matrix = new_vect.fit_transform(test_data_)


In [94]:
predicted_lt = []
predicted_lng = []

for i in range(0,len(test_data_)):
    distances, indices = nbrs.kneighbors(test_c_matrix[i])

    ind = indices[0][0]

    print('ind',ind)
    print(distances)

    lt =train_data.loc[ind,'latitude']
    predicted_lt.append(lt)
    lng = train_data.loc[ind,'longitude']
    predicted_lng.append(lng)
    
    #df_lt = pd.DataFrame([lt], columns=['lt'])
    #df_ln = pd.DataFrame([lng], columns=['ln'])
    
    #data_lt = data_lt.append(df_lt)
    #data_ln = data_ln.append(df_ln)
    #print('test:', test_data.loc[i,'photoTags'])
    #print('train:', train_data.loc[ind,'photoTags'])
    #print('-----------------------------')



ind 2282747
[[ 0.88915361]]
ind 1156385
[[ 0.69812839]]
ind 2004452
[[ 0.98309597]]
ind 29734
[[ 0.72864524]]
ind 2162347
[[ 0.69437033]]
ind 1831915
[[ 0.76438546]]
ind 621665
[[ 0.87052907]]
ind 621665
[[ 0.87052907]]
ind 621665
[[ 0.86229651]]
ind 621665
[[ 0.7501246]]
ind 1029614
[[ 0.74734918]]
ind 5907
[[ 0.92773706]]
ind 217995
[[ 0.90526855]]
ind 121191
[[ 0.]]
ind 1968712
[[ 0.78780344]]
ind 1243638
[[ 0.95567223]]
ind 1715433
[[ 0.89740263]]
ind 1715433
[[ 0.89740263]]
ind 1715433
[[ 0.91375387]]
ind 1715433
[[ 0.91375387]]
ind 1715433
[[ 0.89740263]]
ind 37
[[ 1.]]
ind 37
[[ 1.]]
ind 37
[[ 1.]]
ind 457077
[[ 0.86112489]]
ind 2006232
[[ 0.82013963]]
ind 2006232
[[ 0.82013963]]
ind 2188321
[[ 0.62441856]]
ind 1197402
[[ 0.74067472]]
ind 37
[[ 1.]]
ind 1851283
[[ 0.91693653]]
ind 24935
[[ 0.6513209]]
ind 37
[[ 0.]]
ind 37
[[ 0.]]
ind 37
[[ 0.]]
ind 37
[[ 0.]]
ind 37
[[ 0.]]
ind 37
[[ 0.]]
ind 5000
[[ 0.]]
ind 487949
[[ 0.8084583]]
ind 487949
[[ 0.8084583]]
ind 487949
[[ 0.80845

In [95]:
test_data['lt_predicted'] = predicted_lt
test_data['lng_predicted'] = predicted_lng

In [97]:
test_data.to_csv("predicted_1.csv", encoding="utf-8")

In [104]:
data_predicted = pd.read_csv('predicted_1.csv')
x = data_predicted['count'].sum()
print(x)

1324


In [105]:
accuracy = x/(len(test_data_))

In [106]:
accuracy

0.249811320754717